In [93]:
import os
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [94]:
spark = SparkSession.builder.appName("test").getOrCreate()

In [95]:
pdf_1 = pd.read_csv("/home/rmakin/input/data_1_10.csv")

data_1_10 = spark.createDataFrame(pdf_1)

data_1_10.show(5, False)

+------------+-------------+-----------------+------------------+--------------------+----------------------+----------+
|posting_date|partner_state|legal_entity_name|filing_entity_name|management_unit_name|oracle_gl_product_code|tax_amount|
+------------+-------------+-----------------+------------------+--------------------+----------------------+----------+
|2025-05-03  |OR           |Travelscape, LLC |Expedia, Inc.     |Cheap Tickets       |101003                |34.61     |
|2025-05-02  |WI           |Travelscape, LLC |Expedia, Inc.     |Cheap Tickets       |101003                |132.73    |
|2025-05-01  |WI           |Travelscape, LLC |Expedia, Inc.     |Cheap Tickets       |101004                |29.15     |
|2025-05-02  |NE           |Travelscape, LLC |Expedia, Inc.     |Cheap Tickets       |101008                |21.57     |
|2025-05-03  |CA           |Travelscape, LLC |Travelscape, LLC  |Cheap Tickets       |101003                |12.62     |
+------------+-------------+----

In [96]:
pdf_2 = pd.read_csv("/home/rmakin/input/data_11_20.csv")

data_11_20 = spark.createDataFrame(pdf_2)

data_11_20.show(5, False)

+------------+-------------+---------------------------------+------------------+--------------------+----------------------+----------+
|posting_date|partner_state|legal_entity_name                |filing_entity_name|management_unit_name|oracle_gl_product_code|tax_amount|
+------------+-------------+---------------------------------+------------------+--------------------+----------------------+----------+
|2025-05-12  |NaN          |Expedia Lodging Partner Svcs Sarl|NaN               |Cheap Tickets       |101020                |20.0      |
|2025-05-14  |WA           |Travelscape, LLC                 |Expedia, Inc.     |Cheap Tickets       |101003                |0.0       |
|2025-05-17  |AR           |Travelscape, LLC                 |Expedia, Inc.     |Cheap Tickets       |101004                |10.44     |
|2025-05-15  |CT           |Travelscape, LLC                 |Travelscape, LLC  |Cheap Tickets       |101003                |0.0       |
|2025-05-11  |DC           |Travelscape, 

In [97]:
pdf_3 = pd.read_csv("/home/rmakin/input/data_21_27.csv")

data_21_27 = spark.createDataFrame(pdf_3)

data_21_27.show(5, False)

+------------+-------------+------------------+------------------+--------------------+----------------------+----------+
|posting_date|partner_state|legal_entity_name |filing_entity_name|management_unit_name|oracle_gl_product_code|tax_amount|
+------------+-------------+------------------+------------------+--------------------+----------------------+----------+
|2025-05-27  |Arizona      |Travelscape, LLC  |Travelscape, LLC  |NaN                 |101003                |0.0       |
|2025-05-27  |IA           |Travelscape, LLC  |Travelscape, LLC  |NaN                 |101003                |3633.42   |
|2025-05-27  |CA           |Travelscape, LLC  |Travelscape, LLC  |NaN                 |101009                |4.38      |
|2025-05-21  |NM           |Expedia, Inc. (WA)|Expedia, Inc.     |Cheap Tickets       |101001                |1.33      |
|2025-05-25  |AR           |Travelscape, LLC  |Expedia, Inc.     |Cheap Tickets       |101003                |11.32     |
+------------+----------

In [98]:
data_combined = data_1_10.union(data_11_20).union(data_21_27)

data_combined.show(5, False)

+------------+-------------+-----------------+------------------+--------------------+----------------------+----------+
|posting_date|partner_state|legal_entity_name|filing_entity_name|management_unit_name|oracle_gl_product_code|tax_amount|
+------------+-------------+-----------------+------------------+--------------------+----------------------+----------+
|2025-05-03  |OR           |Travelscape, LLC |Expedia, Inc.     |Cheap Tickets       |101003                |34.61     |
|2025-05-02  |WI           |Travelscape, LLC |Expedia, Inc.     |Cheap Tickets       |101003                |132.73    |
|2025-05-01  |WI           |Travelscape, LLC |Expedia, Inc.     |Cheap Tickets       |101004                |29.15     |
|2025-05-02  |NE           |Travelscape, LLC |Expedia, Inc.     |Cheap Tickets       |101008                |21.57     |
|2025-05-03  |CA           |Travelscape, LLC |Travelscape, LLC  |Cheap Tickets       |101003                |12.62     |
+------------+-------------+----

In [117]:
df_filtered_CA = data_combined.filter("partner_state IN ('CA', 'California')")

In [118]:
df_filtered_CO = data_combined.filter("partner_state IN ('CO', 'Colorado') AND posting_date <= '2025-05-17'")

In [119]:
data_combined_CA_CO = df_filtered_CA.union(df_filtered_CO)

In [120]:
old_data_CA = spark.sql("""
SELECT
  posting_date,
  partner_state,
  legal_entity_name,
  filing_entity_name,
  management_unit_name,
  oracle_gl_product_code,
  SUM(tax_amount) AS tax_amount
FROM egdp_prod_commerce.tax_compliance_eg_reporting_vertex_transaction_fact
WHERE posting_date BETWEEN DATE '2025-05-01' AND DATE '2025-05-27'
  AND product_line = 'Lodging'
  AND partner_state IN ('CA', 'California')
  And partner_country_code = 'USA'
GROUP BY
  posting_date,
  partner_state,
  legal_entity_name,
  filing_entity_name,
  management_unit_name,
  oracle_gl_product_code
  """)

In [121]:
old_data_CO = spark.sql("""
SELECT
  posting_date,
  partner_state,
  legal_entity_name,
  filing_entity_name,
  management_unit_name,
  oracle_gl_product_code,
  SUM(tax_amount) AS tax_amount
FROM egdp_prod_commerce.tax_compliance_eg_reporting_vertex_transaction_fact
WHERE posting_date BETWEEN DATE '2025-05-01' AND DATE '2025-05-17'
  AND product_line = 'Lodging'
  AND partner_state IN ('CO', 'Colorado')
  And partner_country_code = 'USA'
GROUP BY
  posting_date,
  partner_state,
  legal_entity_name,
  filing_entity_name,
  management_unit_name,
  oracle_gl_product_code
  """)

In [122]:
old_data = old_data_CA.union(old_data_CO)

In [123]:
data_combined_CA_CO_alias = data_combined_CA_CO.alias("df1")
old_data_alias = old_data.alias("df2")

data_joined = data_combined_CA_CO_alias.join(
    old_data_alias,
    on=[
        col("df1.posting_date") == col("df2.posting_date"),
        col("df1.partner_state") == col("df2.partner_state"),
        col("df1.legal_entity_name") == col("df2.legal_entity_name"),
        col("df1.filing_entity_name") == col("df2.filing_entity_name"),
        col("df1.management_unit_name") == col("df2.management_unit_name"),
        col("df1.oracle_gl_product_code") == col("df2.oracle_gl_product_code")
    ],
    how="inner"
)

In [125]:
df_final = data_joined.withColumn(
    "tax_amount_difference",
    col("df1.tax_amount") - col("df2.tax_amount")
)

In [128]:
df_result = df_final.select(
    col("df1.posting_date"),
    col("df1.partner_state"),
    col("df1.legal_entity_name"),
    col("df1.filing_entity_name"),
    col("df1.management_unit_name"),
    col("df1.oracle_gl_product_code"),
    col("tax_amount_difference").alias("tax_amount")
)

df_result.show(5, truncate=False)

+------------+-------------+-----------------+------------------+--------------------+----------------------+----------+
|posting_date|partner_state|legal_entity_name|filing_entity_name|management_unit_name|oracle_gl_product_code|tax_amount|
+------------+-------------+-----------------+------------------+--------------------+----------------------+----------+
|2025-05-16  |CA           |Travelscape, LLC |Expedia, Inc.     |Exp Travelocity US  |101003                |0.0       |
|2025-05-16  |CA           |Travelscape, LLC |Travelscape, LLC  |Hcom Finland        |101008                |0.0       |
|2025-05-03  |CA           |Travelscape, LLC |Travelscape, LLC  |Hcom Hong Kong      |101008                |0.0       |
|2025-05-03  |CA           |Travelscape, LLC |Travelscape, LLC  |Exp TAAP Netherlands|101004                |0.0       |
|2025-05-16  |CA           |Travelscape, LLC |Travelscape, LLC  |VRBO                |101008                |0.0       |
+------------+-------------+----

In [132]:
df_result.select(col("tax_amount")).selectExpr("sum(tax_amount) as total").show()

+------------------+
|             total|
+------------------+
|-7736.670000000001|
+------------------+



In [131]:
pdf_4 = df_result.toPandas()

output_dir = "/home/rmakin/output"
os.makedirs(output_dir, exist_ok=True)

pdf_4.to_csv(os.path.join(output_dir, "result.csv"), index=False)